In [1]:
import sys
sys.path.insert(0, "../../python")

import cProfile
from propagation import laser
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from line_profiler import LineProfiler

def do_profile(follow=[]):
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner

In [2]:
Nx = 2**8
X = 100
x = np.linspace(-X/2, X/2, Nx, False)
Ny = 2**8
Y = 100
y = np.linspace(-Y/2, Y/2, Ny, False)

Nz = 256
Z = 100
z = np.linspace(0, Z, Nz)

w0 = 5
zR = np.pi * w0**2
wZ = w0 * np.sqrt(1 + (Z/zR)**2)
E = np.exp(-(np.reshape(x-10, (Nx, 1))**2 + np.reshape(y, (1, Ny))**2) / w0**2)
EE = (w0 / wZ) * np.exp(-(x-10)**2 / wZ**2)

cProfile.run('laser.fourier_prop2(E, x, y, z, 0.8)')

         155 function calls in 1.612 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    1.612    1.612 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:37(_any)
        2    0.000    0.000    0.002    0.001 fftpack.py:102(fft)
        2    0.076    0.038    0.474    0.237 fftpack.py:198(ifft)
        4    0.000    0.000    0.317    0.079 fftpack.py:47(_raw_fft)
        2    0.000    0.000    0.000    0.000 fftpack.py:611(_cook_nd_args)
        2    0.002    0.001    0.478    0.239 fftpack.py:630(_raw_fftnd)
        1    0.000    0.000    0.002    0.002 fftpack.py:834(fft2)
        1    0.000    0.000    0.476    0.476 fftpack.py:924(ifft2)
        4    0.000    0.000    0.000    0.000 fftpack.py:95(_unitary)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:138(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1866(any)
        3    0.0

In [3]:
@do_profile(follow=[laser.fourier_prop2])
def test(E, x, y, z):
    laser.fourier_prop2(E, x, y, z, 0.8)
    
result = test(E, x, y, z)
# I split the function up to see which part was slow, it ends up close to two thirds of the computation is spent
# calculating the np.exp(...) part, potentially can be spead up

Timer unit: 1e-06 s

Total time: 1.38293 s
File: ../../python/propagation/laser.py
Function: fourier_prop2 at line 130

Line #      Hits         Time  Per Hit   % Time  Line Contents
   130                                           def fourier_prop2(E, x, y, z, lam, n=1):
   131                                               """ Propagates an electromagnetic wave from a 2D boundary.
   132                                           
   133                                               Uses the Rayleigh-Sommerfeld transfer function to propagate an
   134                                               electromagnetic wave from a 2D boundary. The calculation assumes a
   135                                               homogeneous index of refraction in the region of propagation.
   136                                           
   137                                               Parameters
   138                                               ----------
   139                              